# Example 1. Compute the transition temperatrue of sh3 based on one bead model

这个例子将展示如何使用openaicg创建一个珠子的粗粒化模型来研究sh3的转变温度$T_{f}$。为了使用该例子在你的本地电脑，你需要从github openaicg2的仓库里将example克隆或者下载到本地。

In [1]:
# import required packages
import openmm as mm
from openmm import app
from openmm import unit
import numpy as np
import pandas as pd
import mdtraj as md
from openaicg2.forcefield.aicgmodel import AICG2Model
from openaicg2 import utils

首先是从pdb文件和psf文件初始化得到topology变量(如果pdb中包括CONNECT部分，就不需要加载psf来完善pdb中topology). 使用utils里的ParserNinfo解析天然信息文件得到力场参数，并存储在ParserNinfo类变量里。

In [7]:
# Initialize
T = 300
tot_simu_steps = 5000
report_period = 1000
friction = 1
timestep = 20

# load pdb and psf files
pdb = md.load('../input/sh3_clementigo.pdb')
psf = md.load_psf('../input/sh3_clementigo.psf')

# complement the bond in topology
bonds = psf._bonds
top = pdb.topology.to_openmm()
rdtop = utils.RedefineTopology()
rdtop.redefine_bond(top,bonds)

# load native information
ParserNinfo=utils.ParserNinfo()
ParserNinfo.get_ninfo('../input/sh3_clementigo.ninfo')

初始化完成后，利用**AICG2Model**类创建一个AICG模型，然后基于模型创建OpenMM的system。具体参数有初始化得到的top, 
use_pbc=True control whether use nonbonded。nonbondedMethod设置nonbonded的性质包括是否截断以及是否使用
周期性边界条件。

In [3]:
model = AICG2Model()
model.create_system(top,use_pbc=True,
                    box_a=100, box_b=100, box_c=100,
                    nonbondedMethod=app.CutoffPeriodic,
                    remove_cmmotion=False)

创建system后，我们利用函数**append_ff_params**输入解析好的力场参数ParserNinfo。对于非键力的计算，除了排除残基索引$|I-J| \le 2$的原子对，
天然接触对也会被排除，由**get_exclusion**函数的exclude_nat_con控制。然后，依次加入各种力。这里需要注意的是最好给每个力设置一个不同的force group,
方便后续可能的力场分析。

In [4]:
# append native information to model
model.append_ff_params(ParserNinfo)
np.save('../output/native_contact.npy',model.protein_intra_contact[['a1','a2','sigma']].to_numpy())
# get exclusion for nonbonded interaction
model.get_exclusion(exclude_nat_con=True)
# add force to system
model.add_protein_bond(force_group=0)
model.add_protein_harmonic_angle(force_group=1)
model.add_protein_native_dihedral(force_group=2)
model.add_protein_native_pair(force_group=3)
model.add_excluded(force_group=4)

get_exclusion
Add protein bonds.
Add protein angles.
Add protein native dihedral angle.
Add protien intra native contact
Add excluded force


利用openMM创建朗之万积分器，然后选定计算平台，例如CPU，构建openMM的simulation变量。

In [5]:
# create a integrator
integrator = mm.LangevinIntegrator(T*unit.kelvin,friction/unit.picosecond,timestep*unit.femtosecond)
init_coord = pdb.xyz[0,:,:] * unit.nanometer
# create a simulation
model.set_simulation(integrator, platform_name='CPU',init_coord=init_coord)
model.move_COM_to_box_center(use_pbc=False)
model.simulation.context.setVelocitiesToTemperature(T*unit.kelvin)
model.simulation.minimizeEnergy()

Use platform: CPU
Move center of mass (COM) to box center.


调用模型继承至simulationsystem的add_reporters函数设置输入输出，tot_simu_steps指模拟的总步长，report_period控制间隔多少记录一次信息，ouput_traj_name设置输出文件路径和名字，report_traj_format设置输出轨迹格式，可选格式有"dcd"和"xtc"，report_traj和report_state_log分别设置是否输出轨迹文件和日志文件。最后，运行模拟。

In [8]:

model.add_reporters(tot_simu_steps, report_period, 
                    output_traj_name='../output/sh3_clementigo',report_traj_format='dcd'
                    ,report_traj=True,report_state_log=True)
print('Running simulation')
model.simulation.step(tot_simu_steps)

Running simulation
